In [1]:
import os

subj = "Subj1"
N_STAGES = 9
exp = "exp_feature_selection"
os.makedirs(f"{subj}/{exp}", exist_ok = True)

In [2]:
%load_ext autoreload
%autoreload 2

import time
import warnings
import itertools
warnings.filterwarnings('ignore')

import SDA
import SDA.analytics
import SDA.clustquality

import umap
import tqdm
import numpy
import pandas
import sklearn.preprocessing
import sklearn.decomposition
import tqdm.contrib.itertools
import sklearn.feature_selection

In [3]:
edges_true = numpy.loadtxt(f"{subj}/reproduction/internal/best_edges.txt").astype(numpy.int32)

In [4]:
params = [ ]

len_st_thr_attempts = [
    [ 0 ],
    [ 20 ],
    [ 40 ],
    [ 60 ],
    # [ 0, 20 ],
    # [ 0, 40 ],
    # [ 0, 60 ],
    # [ 20, 40 ],
    # [ 20, 60 ],
    # [ 40, 60 ],
    # [ 0, 20, 40 ],
    # [ 0, 20, 60 ],
    # [ 0, 40, 60 ],
    # [ 20, 40, 60 ],
    # [ 0, 20, 40, 60 ]
]
for (
    n_clusters_min,
    k_neighbours_min,
    len_st_thr
) in itertools.product(
    range(2, 21, 3), # range(2, 21)
    range(20, 51, 5), # range(20, 51),
    len_st_thr_attempts
):
    for (
        n_clusters_max,
        k_neighbours_max
    ) in itertools.product(
        range(n_clusters_min, 21, 3), # range(n_clusters_min, 21),
        range(k_neighbours_min, 51, 5), # range(k_neighbours_min, 51)
    ):
        k_neighb_max_thr = [ k_neighbours_max ]
        n_cl_max_thr = [ n_clusters_max ]
        
        params.append({
            'scale': False,
            
            'n_clusters_min': n_clusters_min, 'n_clusters_max': n_clusters_max,
            'k_neighbours_min': k_neighbours_min, 'k_neighbours_max': k_neighbours_max,
            'len_st_thr': len_st_thr,

            'n_cl_max_thr': n_cl_max_thr,
            'k_neighb_max_thr': k_neighb_max_thr,
            'n_edge_clusters_min': N_STAGES - 1, 'n_edge_clusters_max': N_STAGES - 1
        })

print(len(params))

3136


In [5]:
N_JOBS = 15

def try_default(features: numpy.ndarray):
    start = time.time()
    target_result, _ = SDA.SDA(n_jobs = N_JOBS, scale = False, verbose = True).apply(features)

    print('Target time:', time.time() - start)
    display(SDA.analytics.best_results(target_result, key = 'Avg-Silh'))

def try_params(features: numpy.ndarray, result_name: str):
    results = [ ]
    for param in tqdm.tqdm(params):
        start = time.time()
        try:
            result, _ = SDA.SDA(**param, n_jobs = N_JOBS, verbose = False).apply(features)
        except Exception as e:
            continue
        end = time.time()

        result = SDA.analytics.best_result(result, key = 'Avg-Silh', n_stages = N_STAGES)
        metrics = SDA.clustquality.cluster_metrics_ground(edges_true, result['St_edges'])
        result['time'] = (end - start)
        results.append(dict(**param, **result, **metrics))
        
    results = pandas.DataFrame(results)
    results.to_csv(f"{subj}/{exp}/{result_name}.csv")
    display(results.head())

### TDA

In [6]:
features_tda = pandas.read_feather(f'{subj}/exp_final_filtered/all_features.feather')
print(features_tda.shape)

features_tda = sklearn.preprocessing.StandardScaler().fit_transform(features_tda)
print(features_tda.shape)

(1046, 3799)
(1046, 3799)


#### UMAP

In [7]:
features_tda_umap = umap.UMAP(n_components = 15, random_state = 42).fit_transform(features_tda)
print(features_tda_umap.shape)

(1046, 15)


In [8]:
try_default(features_tda_umap)

Applying to 1046 samples with 15 features each
Running stage 1


  0%|          | 0/589 [00:00<?, ?it/s]

Running stage 2


  0%|          | 0/672 [00:00<?, ?it/s]

Target time: 37.80610513687134


,St_len_min,K_nb_max,N_cl_max,N_stages,Cl_cen,St_edges,Ward_dist,Cen_dist,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold
0,0,40,10,3,Mode,"[0, 246, 554, 1046]",1035.126427,2.420395,0.165579,612.514876,2.050163,0.284673,479.052251,1.713864
1,0,50,10,4,Mode,"[0, 246, 554, 681, 1046]",1044.994333,3.005025,0.137377,566.736494,2.708340,0.379997,483.041788,0.911399
2,0,40,10,5,Mode,"[0, 246, 554, 682, 789, 1046]",882.027438,2.960657,0.137718,491.752452,2.092489,0.415227,472.130874,0.923173
3,0,35,10,6,Median,"[0, 101, 246, 554, 682, 842, 1046]",591.637795,2.572624,0.058905,458.744853,2.099222,0.367377,317.403342,1.016883
4,0,35,20,7,Median,"[0, 101, 246, 554, 682, 856, 976, 1046]",561.907943,2.643793,0.055811,447.194042,4.757709,0.380541,295.011202,0.954349
5,0,35,15,8,Median,"[0, 101, 246, 554, 682, 789, 856, 976, 1046]",445.311387,2.465364,0.060602,403.837794,4.836544,0.369928,253.451875,1.047580
6,0,35,15,9,Median,"[0, 101, 246, 304, 554, 682, 789, 856, 976, 1046]",298.599010,2.162756,0.060668,362.302324,4.513938,0.327996,180.538247,1.344671
7,0,35,15,10,Median,"[0, 101, 167, 246, 304, 554, 682, 789, 856, 97...",246.765239,1.931269,0.031721,327.643215,4.438314,0.291454,147.799429,1.495095
8,0,35,15,11,Median,"[0, 101, 167, 246, 304, 509, 554, 682, 789, 85...",171.921607,1.752412,0.030197,301.696350,4.447445,0.268016,106.839614,1.687698
9,0,35,10,12,Median,"[0, 101, 167, 246, 304, 509, 554, 682, 789, 84...",152.133663,1.762457,0.002503,281.263004,4.112239,0.259360,98.609576,1.666004


In [9]:
try_params(features_tda_umap, 'tda_umap')

100%|██████████| 3136/3136 [2:27:24<00:00,  2.82s/it]  


,scale,n_clusters_min,n_clusters_max,k_neighbours_min,k_neighbours_max,len_st_thr,n_cl_max_thr,k_neighb_max_thr,n_edge_clusters_min,n_edge_clusters_max,...,Silh,Cal-Har,Dav-Bold,Avg-Silh,Avg-Cal-Har,Avg-Dav-Bold,time,AMI,ARI,FMI
0,False,2,2,20,25,[0],[2],[25],8,8,...,0.055764,400.276960,1.683153,0.193274,68.808055,1.529976,2.212798,0.399592,0.103299,0.376144
1,False,2,2,20,30,[0],[2],[30],8,8,...,0.036657,302.092520,1.942815,0.169087,35.693917,1.662311,1.636544,0.390818,0.094453,0.366944
2,False,2,2,20,35,[0],[2],[35],8,8,...,0.036657,302.092520,1.942815,0.169087,35.693917,1.662311,0.685956,0.390818,0.094453,0.366944
3,False,2,2,20,40,[0],[2],[40],8,8,...,0.026275,241.628393,2.847493,0.169902,19.893034,1.686818,0.891408,0.388644,0.093983,0.366456
4,False,2,2,20,45,[0],[2],[45],8,8,...,0.026275,241.628393,2.847493,0.169902,19.893034,1.686818,1.064749,0.388644,0.093983,0.366456


#### PCA

In [ ]:
features_tda_pca = sklearn.decomposition.PCA(n_components = 15, svd_solver = "full", random_state = 42).fit_transform(features_tda)
print(features_tda_pca.shape)

In [ ]:
try_default(features_tda_pca)

In [ ]:
try_params(features_tda_pca, 'tda_pca')

### Neurofeatures

In [ ]:
df_ft_psd_loc_db = pandas.read_feather(f'{subj}/src/df_ft_psd_loc_db.feather')
df_ft_psd_ind_loc_log = pandas.read_feather(f'{subj}/src/df_ft_psd_ind_loc_log.feather')
df_ft_coh_ind_loc = pandas.read_feather(f'{subj}/src/df_ft_coh_ind_loc.feather')
df_ft_plv_ind_loc = pandas.read_feather(f'{subj}/src/df_ft_plv_ind_loc.feather')

features_neuro = pandas.concat([ df_ft_psd_loc_db, df_ft_psd_ind_loc_log, df_ft_coh_ind_loc, df_ft_plv_ind_loc ], axis = 1)
print(features_neuro.shape)

features_neuro = sklearn.preprocessing.StandardScaler().fit_transform(features_neuro)
print(features_neuro.shape)

#### UMAP

In [ ]:
features_neuro_umap = umap.UMAP(n_components = 15, random_state = 42).fit_transform(features_neuro)
print(features_neuro_umap.shape)

In [ ]:
try_default(features_neuro_umap)

In [ ]:
try_params(features_neuro_umap, 'neuro_umap')

#### PCA

In [ ]:
features_neuro_pca = sklearn.decomposition.PCA(n_components = 15, svd_solver = 'full', random_state = 42).fit_transform(features_neuro)
print(features_neuro_pca.shape)

In [ ]:
try_default(features_neuro_pca)

In [ ]:
try_params(features_neuro_pca, 'neuro_pca')